In [ ]:
%load_ext autoreload
import os
import re
import math
from datetime import datetime
from ltlcross_runner import LtlcrossRunner
from tools_hier import get_tools, ltl3dra_tools, full_tools, tool_order
import tools_hier
from evaluation_utils import sort_by_tools, split_cols
import pandas as pd
import spot
spot.setup()
pd.options.display.max_colwidth = 150

# Comparison of LTL to deterministic automata translators 

$\newcommand{\F}{\mathsf{F}}\newcommand{\G}{\mathsf{G}}\newcommand{\U}{\mathsf{U}}\newcommand{\X}{\mathsf{X}}$
## Tools
We have 
* tools that translate \[fragments\] of LTL into (generalized) Rabin automata:
 - [Rabinizer 3.1](https://www7.in.tum.de/~kretinsk/rabinizer3.html)
 - [Rabinizer 4](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - [LTL3DRA](https://sourceforge.net/projects/ltl3dra/) v.0.2.6 \[LTL$\smallsetminus\G$($\U\X$)\]
* tools that chain translation of LTL into cut-deterministic Büchi automata (aka LDBA) or into deterministic Rabin automata with a construction that create deterministic parity automaton. They avoid Safra-like determinization. Both are from the [owl/Rabinizer4 library](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - ltl2dpa --mode=ldba
 - ltl2dpa --mode=rabinizer (relies on transition-based index appearance record)
* tools that perform Safra-based determinization of N\[G\]BA into {Rabin/Parity} automata that use intermediete NBA
 - [ltl2dstar](http://ltl2dstar.de) \[NBA only\] {Rabin)}
 - autfilt from [Spot](https://spot.lrde.epita.fr/) 2.5 {parity}
 - here we use the following LTL2TGBA/LTL2NBA translators:
   - ltl2tgba from [Spot](https://spot.lrde.epita.fr/) 2.5
   - [LTL3BA](https://sourceforge.net/p/ltl3ba/) v. 1.1.3; we use LTL3BA in two settings:
     - `ltl3ba` {LTL3BA}
     - `ltl3ba -d` {LTL3BAd} which prefer more deterministic automata to smaller ones
 - we further use experimentaly for autfilt the LTL to Emerson-Lei automata translator
   - [LTL3TELA](https://github.com/jurajmajor/ltl3tela) v.1.1.1
* Safra's based determinization into deterministic parity automata that uses information about the LTL formula
 - `ltl2tgba -DG` from [Spot](https://spot.lrde.epita.fr/) 2.5

In [ ]:
!python3 tools_hier.py > tools.tex

In [ ]:
runners = {}
cols=["states","transitions","acc","time","nondet_states"]
for source in ('literature','random'):
    for t in ('full','ltl3dra'):
        name = '{}_{}'.format(source,t)
        tools = full_tools if t=='full' else ltl3dra_tools
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

name = 'random_fg'
tools = ltl3dra_tools
runners[name] = LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

In [ ]:
for name, r in runners.items():
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    r.name = name
    r.parse_results()
    #r.compute_best()
    r.na_incorrect()
    r.orig_count = len(r.values)
    r.clean_count = len(r.values.dropna())
    r.compute_sbacc()

In [ ]:
for name, r in runners.items():
    print('The benchmark {} started with {} formulae and has valid data for {} formulae'.format(name,r.orig_count,r.clean_count))
    r.errors = {}
    for e in ['timeout', 'parse error', 'incorrect', 'crash', 'no output']:
        s = r.get_error_count(e).sum()
        if not pd.isna(s):
            r.errors[e] = r.get_error_count(e).sum()
    s = pd.Series(r.errors, name='count')
    s.index.name = 'error type'
    display(s.reset_index())

## Gather data

In [ ]:
def compute_runner(self,cols=['states','acc','time'],
                   tool_subset=None, split_col_names=True, col_lev_names=None,
                   counts = True):
    '''Returns a dataframe with rows indexed by tools in
    `tool_subset (or `self.tools.keys()`) and columns by
    cols. The values are cummulative sums for respective
    tool and column.'''
    if tool_subset is None:
        tool_subset = self.tools.keys()
    name = self.name
    name = name.replace('ltl3dra','ltl-gux')
    if counts:
        name = '{} ({})'.format(name,self.clean_count)
    col_names = ['{}_{}'.format(name,c) for c in cols]
    df = pd.DataFrame(columns=col_names,index=tool_subset)
    for col in cols:
        df['{}_{}'.format(name,col)] = self.cummulative(col).map(lambda x: '%0.0f' % x)
    if split_col_names:
        res = split_cols(df,axis=1,symbol='_',names=col_lev_names)
    else:
        res = df
    return res 

In [ ]:
display(compute_runner(r).head())
display(compute_runner(r,split_col_names=False).head())
compute_runner(r,col_lev_names=['source','fragment','metric']).head()

In [ ]:
data = []
order = [
    ('literature','full'),
    ('literature','ltl3dra'),
    ('random','full'),
    ('random','ltl3dra'),
    ('random','fg'),
]
for o in order:
    data.append(compute_runner(runners['{}_{}'.format(o[0],o[1])]))
res = pd.concat(data,axis=1)

In [ ]:
def fix_header_colors(lines):
    lines = lines.replace('{c}','{b}',1)
    i = lines.find('{c}')
    return lines[:i+1] + lines[i+1:].replace('{c}','{b}',1)

In [ ]:
def fix_acc(acc,sb=False):
    acc = acc.replace('TEL.TP','TEL.TEL')
    if sb:
        acc = re.sub(r'T([^T]+$)', r'S\1', acc)
    return acc

In [ ]:
def fix_tools(tool):
    tool = tool.replace('ltl2dstar(NBA)',
                        '\parbox[c]{1.3cm}{\centering ltl2dstar NBA}')
    if tool == 'ltl2dstar':
        tool = '\parbox[c]{1.3cm}{\centering ltl2dstar LTL}'
    tool = tool.replace('Spot',
                        #'\rotatebox[origin=c]{90}{Spot (autfilt)}')
                        '\parbox[c]{1.3cm}{\centering Spot autfilt}')
    tool = tool.replace('R3','Rabinizer 3').replace('R4','Rabinizer 4')
    tool = tool.replace('ltl2tgba','Spot')
    return tool

In [ ]:
def high_frag(fr):
    return '\\colorbox{{brown!30}}{{\\hspace{{.5cm}}{}\\hspace{{.5cm}}}}'.format(fr)

In [ ]:
def fix_fragment(fr,vertical=False,color=None):
    fr = fr.replace('full','full~~LTL')
    fr = fr.replace('ltl-gux','\LTLGUX')
    fr = fr.replace('ltl3dra','\LTLGUX')
    fr = fr.replace('fg','\FG').replace('(','[$').replace(')','$]')
    if color:
        source, fr = fr.split('_')
        fr = '\\parbox[c]{{12.9ex}}{{\centering {}}}'.format(fr)
        if source == color:
            fr = high_frag(fr)
    if vertical:
        fr = '\rotatebox[origin=c]{90}{' + fr + '}'
    return fr

In [ ]:
def shorten_index(df, columns=True, cross=False,
                  inplace=False, vertical=False,
                  sb=False, color=None):
    if not inplace:
        df = df.copy()
    if cross:
        i = [(fix_tools(t[0]),t[1],fix_acc(t[2],sb),t[3]) for t in df.index.values]
    else:
        i = [(fix_tools(t[0]),t[1],fix_acc(t[2],sb)) for t in df.index.values]
    df.index=pd.MultiIndex.from_tuples(i)
    if columns:
        if cross:
            vertical = vertical and len(df) > 4
            ci = [(fix_fragment(t[0],vertical,color),t[1]) for t in df.columns.values]
        else:
            ci = [(t[0],fix_fragment(t[1]),t[2]) for t in df.columns.values]
        df.columns=pd.MultiIndex.from_tuples(ci)
    if not inplace:
        return df

In [ ]:
def add_type_lines(filename,Safra=True,vertical=False,end=None):
    cline = '\cmidrule[\lightrulewidth]{2-'+ str(end) +'}' \
        if vertical else '\midrule'
    todo = [(r'(\\multirow\{2\}\{\*\}\{ltl2dpa\})',''),
            (r'(\\multirow\{3\}\{\*\}\{\\parbox\[c\]\{1.3cm\}\{\\centering ltl2dstar LTL\}\})','')]
    if not Safra:
        todo = [todo[0]]
    if vertical:
        todo = [(r"&\s+"+t[0], ' & ') for t in todo]
    for rplc, pref in todo:
        with open(filename) as f:
            lines = f.read()
            lines = re.sub(rplc,cline + pref + r'\1',lines)
        with open(filename,"w") as f1:
            f1.write(lines)

In [ ]:
def fix_lines(filename, end=12, vertical=False):
    end = str(end)
    todo = [(r"(\\multirow\{2\}\{\*\}\{ltl2dpa\})",1,''),
        (r"(& \\multirow\{2\}\{\*\}\{Spot\})",2,'')]
    if vertical:
        todo = [(r"&\s+"+t[0],t[1]+1, ' & ') for t in todo]
    for rplc, start, pref in todo:
        with open(filename) as f:
            lines = f.read()
        lines = re.sub(rplc,
                       "\cmidrule{" +
                       "{}-{}".format(start,end) +
                       "}" + pref + r"\1",
                       lines)
        with open(filename,"w") as f1:
            f1.write(lines)
    with open(filename) as f:
        lines = f.read()
    lines = lines.replace('cline','cmidrule')
    lines = lines.replace('$nan$','---')
    if vertical:
        rplc = "\cmidrule{1-" + str(end) + "}"
        lines = lines.replace(rplc,"\midrule[\heavyrulewidth]")
    # Remove cmidrules in front of midrule
    lines = re.sub(r"(\\cmidrule\{\d+-\d+\}\n?)*\\midrule\[\\heavyrulewidth\](\s*\n*\s*\\cmidrule\{\d+-\d+\}\s*\n?)*", "\\midrule[\\heavyrulewidth] ", lines)
    lines = re.sub(r"(\\cmidrule\{\d+-\d+\}\s*\n?)*(\\cmidrule\[\\lightrulewidth\]\{\d+-\d+\})(\s*\n*\s*\\cmidrule\{\d+-\d+\}\s*\n?)*", r"\2 ", lines)
    with open(filename,"w") as f1:
        f1.write(lines)

In [ ]:
def color_table(filename, extrarowheight='.75ex'):
    setup = '''\\newcolumntype{{a}}{{>{{\\columncolor{{blue!20}}}}r}}
\\newcolumntype{{b}}{{>{{\\columncolor{{blue!20}}}}c}}
\\setlength{{\\aboverulesep}}{{0pt}}
\\setlength{{\\belowrulesep}}{{0pt}}
\\setlength{{\\extrarowheight}}{{{}}}
\\setlength{{\\heavyrulewidth}}{{2pt}}
\\setlength{{\\lightrulewidth}}{{1.2pt}}
\\def\\high{{\\cellcolor{{darkgreen!40}}}}
'''.format(extrarowheight)
    with open(filename) as f:
        lines = f.read()
    lines = fix_header_colors(lines)
    with open(filename,"w") as f1:
        f1.write(setup + lines)

In [ ]:
def make_heading(filename):
    with open(filename) as f:
        lines = f.read()
    lines = re.sub(r"(\s+)&\s+&\s+&\s+states", 
        r"\1main tool & intermediate & acc & states",
        lines)
    lines = re.sub(r"(\s+)&\s+&\s+&\s+&\s+0", 
        r"\1main tool & intermediate & acc & \# & 0",
        lines)
    with open(filename,"w") as f1:
        f1.write(lines)

In [ ]:
def fix_header_colors(lines):
    lines = lines.replace('{c}','{b}',1)
    i = lines.find('{c}')
    return lines[:i+1] + lines[i+1:].replace('{c}','{b}',1)

In [ ]:
def cummulative_to_latex(res,file,transpose=False,color=True):
    if transpose:
        res = res.T
    col_f = 'ccr'
    color_type = 'a' if color else 'r'
    for i in range(len(res.columns)//3):
        if i % 2 == 0:
            col_f += color_type*3
        else:
            col_f += 'rrr'
    res = res
    res.to_latex(buf=open(file,'w'), multirow=True,
         escape=False, na_rep='---',
         float_format=lambda x: '$' + '%0.0f' % x + '$',
         column_format=col_f, multicolumn_format='c')
    if color:
        color_table(file,extrarowheight='.2ex')

In [ ]:
def high_min(data):
    return ['\high ${:0.0f}$'.format(m) if 
            m == data.min() else m for m in data]

def high_max(data):
    return ['\high ${:0.0f}$'.format(m) if 
            m == data.max() else m for m in data]

In [ ]:
res = sort_by_tools(res,tool_order)
res = split_cols(res,axis=0,symbol='/')
shorten_index(res,inplace=True)
sorted_all = res.astype(float).apply(high_min)

In [ ]:
cumulative = [('random','rand'),('literature','lit')]
for long, short in cumulative:
    filename = 'cum_{}.tex'.format(short)
    cummulative_to_latex(sorted_all[long], file=filename)
    # We need to measure the # of columns in the LaTeX table
    length = len(sorted_all[long].columns) +\
             len(sorted_all[long].index.levels)
    fix_lines(filename,length)
    add_type_lines(filename)
    make_heading(filename)

## Errors

In [ ]:
def color_str_multiindex_cols(df, color_type='a', def_type='r'):
    col_f = '' # string to acummulate the columns' types
    i = 0 # iterates over labels
    c = 0 # remembers if this one should be colored
    while i < len(df.columns):
        col = df.columns.labels[0][i]
        current_lab = df.columns.levels[0][col]
        col_nums = len(df[current_lab].columns)
        i += col_nums # moves to next column
        if c % 2 == 0:
            col_f += color_type*col_nums
        else:
            col_f += def_type*col_nums
        if len(df[current_lab]) > 0:
            c += 1
    return col_f

def multicol_single(df, col_type='c'):
    df = df.copy()
    def multicol(col):
        return '\\multicolumn{{1}}{{{}}}{{{}}}'.format(col_type, col)
    i = 0 # iterates over labels
    create_multicol = []
    while i < len(df.columns):
        col = df.columns.labels[0][i]
        current_lab = df.columns.levels[0][col]
        col_nums = len(df[current_lab].columns)
        if len(df[current_lab].columns) == 1:
            create_multicol.append(current_lab)
        i += col_nums # moves to next column
    ci = [multicol(col) if col in create_multicol else col for 
          col in df.columns.levels[0]]
    df.columns.set_levels(ci,level=0,inplace=True)
    return df

In [ ]:
def errors_to_latex(res,file,color=True):
    col_f = 'l'
    if color:
        col_f += color_str_multiindex_cols(res,'a')
    else:
        col_f += 'r'*len(res.columns)
    res = multicol_single(res)
    res = res.dropna(how='all')
    res.to_latex(buf=open(file,'w'), multirow=True,
         escape=False, na_rep='---',
         float_format=lambda x: '$' + '%0.0f' % x + '$',
         column_format=col_f, multicolumn_format='c')
    if color:
        color_table(file)
    fix_lines(file)

In [ ]:
names = ['literature_full',
 'literature_ltl3dra',
 'random_full',
 'random_ltl3dra',
 'random_fg']
data = []
for name in names:
    r = runners[name]
    errors = {}
    for e in ['timeout', 'parse error', 'incorrect', 'crash', 'no output']:
        s = pd.DataFrame(r.get_error_count(e))
        if not s.empty:
            errors[e] = s.unstack(level=0)
    if len(errors) > 0:
        res = pd.concat(errors,axis=1)
        res.index = res.index.droplevel(0)
        data.append(res)
errors = pd.concat(data,keys=names,axis=1)
ci = [(t[0].split('_')[0],
       fix_fragment(t[0].split('_')[1]),
       t[1].replace('parse error', '>32 marks')) for t in errors.columns.values]
errors.columns = pd.MultiIndex.from_tuples(ci)
errors.index = [tools_hier.fix_tool(t) for t in errors.index.values]

In [ ]:
file = 'errors_lit.tex'
errors_to_latex(errors.literature,file)
file = 'errors_rand.tex'
errors_to_latex(errors.random,file)

## Cross-comparison

A tool `t1` wins against `t2` in the comparison also if `t2` fails and `t1` does not.

In [ ]:
def prepare_cross(df,victories=True):
    df = split_cols(df,axis=0,symbol='/')  
    df = df.T.reset_index(drop=True).T
    df = df.reset_index()
    df = df.set_index(['level_0','level_1','level_2'],
                      append=True)
    df = df.reorder_levels([1,2,3,0])
    df.index.names = ['','','','']
    if victories:
        cols = list(df.columns)
        cols[len(cols)-1] = 'V'
        df.columns = cols
    return df

In [ ]:
def cross_to_latex(res, file='cross.tex', color=True,
                   vertical=False):
    col_f = 'cccrr' if vertical else 'ccrr'
    color_type = 'a' if color else 'r'
    for i in range(len(res.columns)):
        if i % 2 == 0:
            col_f += color_type
        else:
            col_f += 'r'
    res = res
    res.to_latex(buf=open(file,'w'), multirow=True,
        escape=False,na_rep='---', 
        float_format=lambda x: '$' + '%0.0f' % x + '$',
        column_format=col_f,multicolumn_format='c')

In [ ]:
def create_cross(runners_base, tools, 
                 fragments=['full','ltl3dra'],
                 cols=['states','acc'],
                 vertical=False, merge_in_latex=True,
                 filename='cross.tex', latex=True,
                 counts=True):   
    def to_latex(table, filename=filename, vertical=False):
        table = table.copy()
        if vertical:
            data = []
            for k in table.index.levels[0]:
                data.append(
                    pd.DataFrame(table.loc[k,'V']).\
                    apply(high_max))
                vict = pd.concat(data,
                    keys=table.index.levels[0])
            table.loc[:,'V'] = vict
        else:
            table.loc(axis=1)[:,'V'] = \
                table.loc(axis=1)[:,'V'].apply(high_max)
        cross_to_latex(table, file=filename, 
                       vertical=vertical)
        end = len(table.columns) + len(table.index.levels)
        add_type_lines(filename, Safra=False, 
                       vertical=vertical, end=end)
        fix_lines(filename, end, vertical)
        color_table(filename)
        make_heading(filename)
    
    state_based = 'sb_states' in cols
    data = []
    fr_and_c = []
    for fr in fragments:
        r = runners['{}_{}'.format(runners_base,fr)]
        fr_and_c.append('{} ({})'.format(fr,r.orig_count))
        res = r.cross_compare(tools=tools,props=cols)
        cross = prepare_cross(res)
        data.append(cross)
        if latex and not merge_in_latex:
            table = pd.concat([cross], axis=1, keys=[fr])
            table = shorten_index(table,cross=True,
                        columns=False,inplace=False,
                        sb=state_based)
            fr_name = '{}_{}.tex'.format(filename[:-4],fr)
            to_latex(table, fr_name)
    if counts:
        fragments = fr_and_c
    table = pd.concat(data, axis=1, keys=fragments)
    shorten_index(table, cross=True, inplace=True,
                  vertical=vertical,
                  sb=state_based)
    if vertical:
        fragments = [fix_fragment(f, vertical = 
                     len(table) > 4)
                     for f in fragments]
        data = [table[f] for f in fragments]
        table = pd.concat(data,keys=fragments)
    if latex and merge_in_latex:
        to_latex(table,filename,vertical)
    return table

In [ ]:
def create_cross_merged(tools, 
                 cols=['states','acc'],
                 vertical=False, merge_in_latex=True,
                 filename='cross_merged.tex', latex=True,
                 counts=True, color='random'):
    def to_latex(table, filename=filename, vertical=False):
        table = table.copy()
        if vertical:
            data = []
            for k in table.index.levels[0]:
                data.append(
                    pd.DataFrame(table.loc[k,'V']).\
                    apply(high_max))
                vict = pd.concat(data,
                    keys=table.index.levels[0])
            table.loc[:,'V'] = vict
        else:
            table.loc(axis=1)[:,'V'] = \
                table.loc(axis=1)[:,'V'].apply(high_max)
        cross_to_latex(table, file=filename, 
                       vertical=vertical)
        end = len(table.columns) + len(table.index.levels)
        add_type_lines(filename, Safra=False, 
                       vertical=vertical, end=end)
        fix_lines(filename, end, vertical)
        color_table(filename)
        make_heading(filename)
    
    state_based = 'sb_states' in cols
    data = []
    fr_and_c = []
    r_order = ['literature_full','literature_ltl3dra',
               'random_full','random_ltl3dra','random_fg']
    for name in r_order:
        r = runners[name]
        source, fr = name.split('_')
        fr_and_c.append('{} ({})'.format(name,r.orig_count))
        res = r.cross_compare(tools=tools,props=cols)
        cross = prepare_cross(res)
        data.append(cross)
    if counts:
        fragments = fr_and_c
    table = pd.concat(data, axis=1, keys=fragments)
    shorten_index(table, cross=True, inplace=True,
                  vertical=vertical,
                  sb=state_based,
                  color=color)
    if vertical:
        fragments = [fix_fragment(f, vertical = 
                     len(table) > 4, color=color)
                     for f in fragments]
        data = [table[f] for f in fragments]
        table = pd.concat(data,keys=fragments)
    if latex and merge_in_latex:
        to_latex(table,filename,vertical)
    return table

In [ ]:
direct = [t for t in tool_order if
          'LTL3DRA' in t or
          'R3' in t or 'R4' in t]
tgr = [t for t in tool_order if
       'TGR' in t]
ltl2dstar = [t for t in tool_order if 'ltl2dstar' in t]
spot = [t for t in tool_order if 
        t.startswith('Spot') or
        'ltl2tgba' in t]
ltl2dpa = ['ltl2dpa/ldba/TP','ltl2dpa/Rab/TP']
Safra = ltl2dstar + spot
selection = [t for t in tool_order if
             'R4//TGR' in t or
             'ltl2dpa' in t or
             #'ltl2dstar/Spot' in t or
             'ltl2tgba' in t
]
toolsets = {
    'direct' : direct,
    'ltl2dstar' : ltl2dstar,
    'spot' : spot,
    'selection' : selection
    #'parity' : parity,
    #'best' : best
}
toolsets_sbacc = {
    #'Safra_sel' : Safra_sel,
    #'Rabin' : Rabin,
    #'selection_sr' : selection_sr
}

In [ ]:
for name, tools in toolsets.items():
    filename = 'cross_m_{}.tex'.format(name)
    create_cross_merged(tools, 
                 filename=filename, merge_in_latex=True,
                 vertical=True)
for name, tools in toolsets_sbacc.items():
    filename = 'cross_m_{}.tex'.format(name)
    create_cross_merged(tools, cols=['sb_states','acc'],
                 filename=filename, merge_in_latex=True,
                 vertical=True)

## Bar plots for minimal automata counts

In [ ]:
approaches = {
    'direct' : direct,
    'Safra'  : Safra,
    'ltl2dpa': ltl2dpa    
}
app_order = ['direct','Safra','ltl2dpa']
fr_order = ['full','ltl3dra','fg']

In [ ]:
for r in runners.values():
    for ap_name, tools in approaches.items():
        r.compute_best(tools, ap_name)

In [ ]:
def get_min_data(runners, tools, restrict_tools=False, unique_only=False,col='states'):
    '''
    runners : list of runners
    tools : list of tools in their order
    '''
    data = {}
    if isinstance(runners,dict):
        runners = runners.values()
    for r in runners:
        res = r.min_counts(tools, restrict_tools=restrict_tools,
                           unique_only=unique_only,col=col)
        if unique_only:
            nu = pd.DataFrame(index=['nonunique'],columns=[0])
            nu[0]=[r.orig_count-res.sum()[0]]
            res = res.append(nu)
        ## sets tool from index to a columns
        res.index.name = 'tool'
        res.reset_index(inplace=True)
        r.orig_c_name = '{} [{}]'.format(r.name, r.orig_count)
        data[r.orig_c_name] = res
    ret = pd.concat(data, names = ['benchmark'])
    ret = ret.reset_index(drop=True,level=1)
    df = split_cols(ret,'_',axis=0,names=['source','fragment'])
    df = df.reset_index().set_index(['source','fragment','tool'])
    df.index = df.index.droplevel(0)
    df = df.unstack().swaplevel(axis=1)
    df.columns = df.columns.droplevel(1)
    return df

In [ ]:
def dict_to_plot(d, tools, alternate_labels=False):
    frags = []
    for f in fr_order:
        for rf in d[tools[0]].keys():
            if rf.startswith(f):
                frags.append(rf)
    alt_str = 'nodes near coords align={below=10pt, anchor=north}'
    res = ''
    i = 0
    for tool in tools:
        vals = d[tool]
        coords = ['({},{})'.format(vals[fr],fr) if not math.isnan(vals[fr]) else 
                  '(0,{})'.format(fr) for fr in frags]
        if not alternate_labels or i % 2 == 0:
            res += '\\addplot coordinates {{{}}};\n'.format(' '.join(coords))
        else:
            res += '\\addplot+[{}] coordinates {{{}}};\n'.format(alt_str,' '.join(coords))
        res += '\\addlegendentry{{{}}}\n'.format(tool)
        i += 1
    return res

In [ ]:
def min_bar_plot(df,tools,filename=None,kw=None):
    d = df.to_dict()
    fr = []
    for f in fr_order:
        for rf in d[tools[0]].keys():
            if rf.startswith(f):
                fr.append(rf)
    fr.reverse()

    #Process kw args
    if kw is not None:
        args = ['{}={},\n'.format(k,v) for k,v in kw.items()]
        args = ''.join(args)
    else:
        args = ''

    coords = dict_to_plot(d,tools)
    res = '''\\begin{{tikzpicture}}[scale=.83]
\\pgfplotsset{{compat=1.14}}
\\pgfplotsset{{every axis legend/.append style=
{{nodes={{text width=2cm, inner xsep=5pt,text depth=0.15em}}}}}}
\\begin{{axis}}[
xbar=-22pt, xmin=0,
axis x line* = bottom,
axis y line* = left,
width=12cm, height={}cm, enlarge y limits={},
xlabel={{\\# of automata with minimal size}},
symbolic y coords={{{}}},
ytick={{{}}},
yticklabels={{{}}},
yticklabel style={{align=center, text width=1.95cm}},
nodes near coords, nodes near coords align={{horizontal}},
legend style={{at={{(0.5,1.05)}},
    anchor=south,legend columns=-1,
    }},
{}%
]
{}%
\\end{{axis}}
\\end{{tikzpicture}}'''.format(2.5*len(fr),
                               1/len(fr),
                               ','.join(fr),
                               ','.join(fr),
                               ','.join([fix_fragment(f) for f in fr]),
                               args,
                               coords)
    if filename is not None:
        print(res,file=open(filename,'w'))
    else: 
        return res

In [ ]:
def min_stacked_bar_plot(df,tools,filename=None,alternate=False,kw=None):
    d = df.to_dict()
    fr = []
    for f in fr_order:
        for rf in d[tools[0]].keys():
            if rf.startswith(f):
                fr.append(rf)
    fr.reverse()

    #Process kw args
    if kw is not None:
        args = ['{}={},\n'.format(k,v) for k,v in kw.items()]
        args = ''.join(args)
    else:
        args = ''

    coords = dict_to_plot(df.to_dict(),tools, alternate)
    res = '''\\begin{{tikzpicture}}[scale=.81]
\\pgfplotsset{{compat=1.14}}
\pgfplotsset{{every axis legend/.append style=
{{nodes={{text width=1.6cm, inner xsep=5pt,text depth=0.15em}}}}}}
\\begin{{axis}}[
xbar stacked, xmin=0,
bar width=20pt,
axis x line* = bottom,
axis y line* = left,
width=12cm, height={}cm, enlarge y limits={},
xlabel={{\\# of automata with minimal size}},
symbolic y coords={{{}}},
ytick={{{}}},
yticklabels={{{}}},
yticklabel style={{align=center, text width=1.95cm}},
nodes near coords, nodes near coords align={{above=10pt, anchor=south}},
legend style={{at={{(0.5,1.05)}},
    anchor=south,legend columns=-1}},
{}%
]
{}%
\\end{{axis}}
\\end{{tikzpicture}}'''.format(2*len(fr)+1.5,
                               1/len(fr),
                               ','.join(fr),
                               ','.join(fr),
                               ','.join([fix_fragment(f) for f in fr]),
                               args,
                               coords)
    if filename is not None:
        print(res,file=open(filename,'w'))
    else: 
        return res

In [ ]:
def min_bar(runners, tools, filename=None,kw=None):
    return min_bar_plot(get_min_data(runners,tools),
                        tools,filename,kw=kw)

In [ ]:
def min_stacked(runners, tools, filename=None, alternate=False,kw=None):
    df =  get_min_data(runners,tools,restrict_tools=True,unique_only=True)
    return min_stacked_bar_plot(df, tools+['nonunique'], 
                                filename, alternate,kw=kw)

In [ ]:
lit = [r for r in runners.values() if 'lit' in r.name]
rand = [r for r in runners.values() if 'random' in r.name]

In [ ]:
second = {'legend to name' : 'nope'}
min_bar(lit,app_order,'min_bar_lit.tex')
min_bar(rand,app_order,'min_bar_rand.tex',kw=second)

In [ ]:
min_stacked(lit,app_order,'min_st_bar_lit.tex')
min_stacked(rand,app_order,'min_st_bar_rand.tex',True,kw=second)

In [ ]:
get_min_data(runners.values(),['ltl2tgba//TP','Spot/LTL3TELA/TEL.TP'],unique_only=True)

### Inverse plot for tools

In [ ]:
def dict_to_plot_inv(d, tools, alternate_labels=False):
    frags = []
    for f in fr_order:
        for rf in d:
            if rf.startswith(f):
                frags.append(rf)
    to = [t for t in tool_order if t in tools]
    alt_str = 'nodes near coords align={below=10pt, anchor=north}'
    res = ''
    i = 0
    for f in frags:
        vals = d[f]
        coords = ['({},{{{}}})'.format(vals[tool],tool) if not math.isnan(vals[tool]) else 
                  '(0,{{{}}})'.format(tool) for tool in to]
        if not alternate_labels or i % 2 == 0:
            res += '\\addplot coordinates {{{}}};\n'.format(' '.join(coords))
        else:
            res += '\\addplot+[{}] coordinates {{{}}};\n'.format(alt_str,' '.join(coords))
        res += '\\addlegendentry{{{}}}\n'.format(fix_fragment(f))
        i += 1
    return res

In [ ]:
def min_bar_plot_inv(df,tools,filename=None,kw=None):
    d = df.to_dict()
    fr = []
    for f in fr_order:
        for rf in d:
            if rf.startswith(f):
                fr.append(rf)
    fr.reverse()
    tools = tools.copy()
    tools.reverse()
    coords = dict_to_plot_inv(d,tools)
    if kw is not None:
        args = ['{}={},\n'.format(k,v) for k,v in kw.items()]
        args = ''.join(args)
    else:
        args = ''
    res = '''\\begin{{tikzpicture}}
\\pgfplotsset{{compat=1.14}}
\\pgfplotsset{{every axis legend/.append style=
{{nodes={{inner xsep=5pt,
text depth=,align=center}}}}}}
\\begin{{axis}}[
xbar=-12pt, xmin=0,
bar width=5pt,
axis x line* = bottom,
axis y line* = left,
width=6.8cm, height={}cm, enlarge y limits={},
xlabel={{\\# of automata with minimal size}},
symbolic y coords={{{}}},
ytick={{{}}},
yticklabels={{{}}},
yticklabel style={{align=left,font=\\small}},
nodes near coords, nodes near coords align={{horizontal}},
nodes near coords style={{font=\\small}},
legend style={{at={{(0.5,1.05)}},
    anchor=south,legend columns=-1,
    }},
{}%
]
{}%
\\end{{axis}}
\\end{{tikzpicture}}'''.format(len(tools),
                               1/len(tools),
                               ','.join(tools),
                               ','.join(tools),
                               ','.join([tools_hier.fix_tool(t) for t in tools]),
                               args,
                               coords)
    if filename is not None:
        print(res,file=open(filename,'w'),end='%')
    else: 
        return res

In [ ]:
def min_bar_inv(runners, tools, filename=None,col='states',kw=None):
    return min_bar_plot_inv(get_min_data(runners,tools,col=col).T,tools,filename,kw=kw)

In [ ]:
first_l = {'legend to name' : 'leg_min_l'}
first_r = {'legend to name' : 'leg_min_r'}
marks = {'title' : 'mixed marks'}
first_l.update(marks)
first_r.update(marks)
second = {'yticklabels' : '', 'legend to name' : 'xxx', 'title': 'marks on states'}
min_bar_inv(lit,tool_order,'min_bar_lit_inv.tex',kw=first_l)
min_bar_inv(lit,tool_order,'min_bar_lit_sbacc_inv.tex',col='sb_states', kw=second)
min_bar_inv(rand,tool_order,'min_bar_rand_inv.tex',kw=first_r)
min_bar_inv(rand,tool_order,'min_bar_rand_sbacc_inv.tex',col='sb_states', kw=second)

## Sorted plots

In [ ]:
def sorted_plot(self,tools,filename=None,col='states',short=False):
    tool_coord = '''\\addplot coordinates {{{}}};%
\\addlegendentry{{{}}}%
'''
    sh_width = '11cm'
    width = sh_width if short else '16cm'
    v = r.values[col].loc(axis=1)[tools].dropna()
    coords_str = ''
    for tool in tools:
        values = sorted(list(v[tool]))
        t_coords = ['({},{})'.format(i,values[i]) for i in range(len(values))]
        t_coords_str = ' '.join(t_coords)
        coords_str += tool_coord.format(t_coords_str,tools_hier.fix_tool(tool))
    res = '''\\begin{{tikzpicture}}
\\pgfplotsset{{every axis legend/.append style={{
cells={{anchor=west}},
draw=none,
}}}}
\\pgfplotsset{{compat=1.14}}
\\begin{{semilogyaxis}}[
xmin=0,xmax={},
thick,
no markers,
cycle list name=linestyles*,
axis x line* = bottom,
axis y line* = left,
width={}, height=7cm, 
xlabel={{$n$-th smallest automaton}},
ylabel={{states}},
legend pos = north west,
cycle list={{%
{{darkgreen, solid}},
{{blue, densely dashed}},
{{red, dashdotdotted}},
{{black, densely dotted}},
{{brown, loosely dashdotted}}
}}
]
{}
\\end{{semilogyaxis}}
\\end{{tikzpicture}}'''.format(len(self.values),
                               width,
                               coords_str)
    if filename is not None:
        print(res,file=open(filename,'w'))
    else: 
        return res

In [ ]:
t = ['Spot/LTL3TELA/TEL.TP','Spot/LTL3BA/TGB.TP','ltl2tgba//TP']
min_name = 'min(Spot)'
for name in ['random_fg','random_ltl3dra']:
    r = runners[name]
    r.compute_best(spot,min_name)
    sorted_plot(r,[min_name]+t,'quantile_ltl3tela_{}.tex'.format(name))

## Scatter plots for non-equal

In [ ]:
def sc_plot(r,t1,t2,filename=None,include_equal = True,col='states',log=None,size=(7,6.5),kw=None,clip=None, add_count=True):
    merged = isinstance(r,list)
    if merged:
        vals = pd.concat([run.values[col] for run in r])
        vals.index = vals.index.droplevel(0)
        vals = vals.groupby(vals.index).first()
    else:
        vals = r.values[col]
    to_plot = vals.loc(axis=1)[[t1,t2]] if include_equal else\
        vals[vals[t1] != vals[t2]].loc(axis=1)[[t1,t2]]
    to_plot['count'] = 1
    to_plot.dropna(inplace=True)
    to_plot = to_plot.groupby([t1,t2]).count().reset_index()
    if filename is not None:
        print(scatter_plot(to_plot, log=log, size=size,kw=kw,clip=clip, add_count=add_count),file=open(filename,'w'))
    else:
        return scatter_plot(to_plot, log=log, size=size,kw=kw,clip=clip, add_count=add_count)

In [ ]:
def scatter_plot(df, short_toolnames=True, log=None, size=(7,6.5),kw=None,clip=None,add_count = True):
    t1, t2, _ = df.columns.values
    if short_toolnames:
        t1 = fix_tools(t1.split('/')[0])
        t2 = fix_tools(t2.split('/')[0])
    vals = ['({},{}) [{}]\n'.format(v1,v2,c) for v1,v2,c in df.values]
    plots = '''\\addplot[
    scatter, scatter src=explicit, 
    only marks, fill opacity=0.5,
    draw opacity=0] coordinates
    {{{}}};'''.format(' '.join(vals))
    start_line = 0 if log is None else 1
    line = '\\addplot[darkgreen,domain={}:{}]{{x}};'.format(start_line, min(df.max(axis=0)[:2])+1)
    axis = 'axis'
    mins = 'xmin=0,ymin=0,'
    clip_str = ''
    if clip is not None:
        clip_str = '\\draw[red,thick] ({},{}) rectangle ({},{});'.format(*clip)
    if log:
        if log == 'both':
            axis = 'loglogaxis'
            mins = 'xmin=1,ymin=1,'
        else:
            axis = 'semilog{}axis'.format(log)
            mins = mins + '{}min=1,'.format(log)
    args = ''
    if kw is not None:
        if 'title' in kw and add_count:
            kw['title'] = '{{{} [{}]}}'.format(kw['title'],df['count'].sum())
        args = ['{}={},\n'.format(k,v) for k,v in kw.items()]
        args = ''.join(args)
    res = '''%\\begin{{tikzpicture}}
\\pgfplotsset{{every axis legend/.append style={{
cells={{anchor=west}},
draw=none,
}}}}
\\pgfplotsset{{colorbar/width=.3cm}}
\\pgfplotsset{{title style={{align=center,
                        font=\\small}}}}
\\pgfplotsset{{compat=1.14}}
\\begin{{{0}}}[
{1}
colorbar,
%thick,
axis x line* = bottom,
axis y line* = left,
width={2}cm, height={3}cm, 
xlabel={{{4}}},
ylabel={{{5}}},
cycle list={{%
{{darkgreen, solid}},
{{blue, densely dashed}},
{{red, dashdotdotted}},
{{brown, densely dotted}},
{{black, loosely dashdotted}}
}},
{6}%
]
{7}%
{8}%
{9}%
\\end{{{0}}}
%\\end{{tikzpicture}}
'''.format(axis,mins,
                    size[0],size[1],t1,t2,
                    args,plots,line,
                    clip_str)
    return res

In [ ]:
ltl2tgba = 'ltl2tgba//TP'
ltl2dstar = 'ltl2dstar/Spot/SB.SR'
margin_size = (4.3,6)
clip_names = ('xmin','ymin','xmax','ymax')

In [ ]:
size = (4.75,6)
clips = {
    'full' : (0,0,33,33),
    'ltl3dra' : (0,0,22,22),
}
fr_runs = {
    'full' : [r for name, r in runners.items() if 'full' in name],
    'ltl3dra' : [r for name, r in runners.items() if 'ltl3dra' in name],
    'fg' : runners['random_fg']
}

In [ ]:
for name, r in fr_runs.items():
    fr = fix_fragment(name)
    clip = None
    kw={'title' : fr}
    # reuse the y axis labels from previous
    if name in ['ltl3dra','fg']:
        kw['ylabel'] = ''
    if name in clips:
        clip = clips[name]
        cl_kw = kw.copy()
        cl_kw['title'] = '\\\\zoom'
        cl_kw.update(zip(clip_names,clip))
    saf_kw = kw.copy()
    # reuse the y axis labels from zooms
    # and make the one without zoom overlayed
    #if name in ['ltl3dra','full']:
    #    kw['xlabel'] = ''
    #else:
    #    kw['xlabel style'] = '{overlay}'
    sc_plot(r,'R4//TGR',ltl2tgba,'sc_best_{}.tex'.format(name),size=size,kw=kw,clip=clip)
    if clip is not None:
        sc_plot(r,'R4//TGR',ltl2tgba,'sc_best_clip_{}.tex'.format(name),size=size,kw=cl_kw,add_count=False)
    
    ### Safra: plots for ltl2dstar and Spot
    saf_kw['title'] = 'marks on states\\\\' + saf_kw['title']
    if 'ylabel' not in saf_kw:
        saf_kw['ylabel'] = 'ltl2dstar (LTL)'
    sc_plot(r,ltl2tgba,ltl2dstar,'sc_Saf_{}.tex'.format(name),
            col='sb_states',log='both',size=size,
            kw=saf_kw)

size = (4.6,6)
ltl3tela_kw = {'ylabel' : 'Spot + LTL3TELA'}
ltl3tela_clip = (0,0,5,20)
sc_plot(runners['random_fg'],'R4//TGR',
        'Spot/LTL3TELA/TEL.TP','sc_ltl3tela.tex',
        size=size,clip=ltl3tela_clip,kw=ltl3tela_kw)
ltl3tela_kw.update(zip(clip_names,ltl3tela_clip))
ltl3tela_kw['ylabel'] = ''
ltl3tela_kw['title'] = 'zoom'
sc_plot(runners['random_fg'],'R4//TGR',
        'Spot/LTL3TELA/TEL.TP','sc_clip_ltl3tela.tex',
        size=size,kw=ltl3tela_kw,add_count=False)